In [ ]:
# Jag antar att tensorflow och tensorflow_hub redan är installerade (lektion 5)
#!pip install tensorflow
#!pip install tensorflow_hub

In [ ]:
# Alla nödvändiga bibliotek
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Rectangle
from PIL import Image
from pathlib import Path

In [ ]:
# Vår bild
image = Image.open(Path("data") / "jakan.JPG")
plt.axis('off')
plt.imshow(image)

In [ ]:
# En enkel object_detect
def simple_object_detect(content_path):
     # Load content image
    content_image = plt.imread(content_path)
    # Convert to float32 numpy array, add batch dimension, and normalize to range [0, 1]. Example using numpy:
    # Det är samma krav på bilden som för style_transfer (lektion 6)
    content_image = content_image.astype(np.float32)[np.newaxis, ...] / 255.

    # Detect
    # Problem med anropet därför signatures['default']
    detector_output = detector.signatures['default'](tf.constant(content_image))
    return(detector_output)

In [ ]:
# Om fel "tf hub OSError: SavedModel file does not exist at" ta bort foldern och kör om...

# Ladda detektorn från TensorFlow Hub
detector = hub.load("https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1")

In [ ]:
output = simple_object_detect(Path("data") / "jakan.JPG")
print(output)

## output dictionary

In [ ]:
# output är ett dictionary {}
# vi är bara instresserade detection_scores, detection_class_entities och detection_boxes
scores = output['detection_scores']
classes = output['detection_class_entities']
boxes = output['detection_boxes']
# gör om från tensorer till numpy arrayer
scores = scores.numpy()
# gör oxå om från byte till string
classes = classes.numpy().astype(str)
boxes = boxes.numpy()

for i in range(len(scores)):
    # skriv ut om score >= 0.2
    if (scores[i] >= 0.20):
        print(str(i), classes[i], scores[i], boxes[i])

## plotta bilden och objekten

In [ ]:
# plotta bilden med bounding boxes och objekt-nummer
image = Image.open(Path("data") / "jakan.JPG")
# Set the size of the plot figure
plt.figure(figsize=(12, 12))
plt.imshow(image)
plt.axis('off')
# Get the current reference
ax = plt.gca()

for i in range(5):
    ymin = boxes[i][0]
    xmin = boxes[i][1]
    ymax = boxes[i][2]
    xmax = boxes[i][3]
    # Create a Rectangle patch
    width = image.width*(xmax - xmin)
    height = image.height*(ymax - ymin)
    rect = Rectangle((image.width*xmin, image.height*ymin), width, height, 
                     linewidth=2, edgecolor='r', facecolor='none')
    plt.text(image.width*xmin, image.height*ymin, str(i), color="red", 
             fontdict={"fontsize":20,"fontweight":'bold',"ha":"left", "va":"baseline"})
    # Add the patch to the Axes
    ax.add_patch(rect)
    
plt.show()

# Funktionen object_detect()

In [ ]:
def object_detect(content_path, score_min):
    # Load content image
    content_image = plt.imread(content_path)
    # Convert to float32 numpy array, add batch dimension, and normalize to range [0, 1]. Example using numpy:
    # Det är samma krav på bilden som för style_transfer (lektion 6)
    content_image = content_image.astype(np.float32)[np.newaxis, ...] / 255.

    # Detect
    # Problem med anropet därför signatures['default']
    output = detector.signatures['default'](tf.constant(content_image))
    
   # output är ett dictionary {}
    # vi är bara instresserade detection_scores, detection_class_entities och detection_boxes
    scores = output['detection_scores']
    classes = output['detection_class_entities']
    boxes = output['detection_boxes']
    # gör om från tensorer till numpy arrayer
    scores = scores.numpy()
    # gör oxå om från byte till string
    classes = classes.numpy().astype(str)
    boxes = boxes.numpy()
    
    for i in range(len(scores)):
        # skriv ut om score >= score_min
        if (scores[i] >= score_min):
            print(str(i), classes[i], scores[i])
            num_obj = i

    # plotta bilden med bounding boxes och objekt-nummer
    edge = ['r', 'y', 'b', 'g']
    image = Image.open(content_path)
    # Set the size of the plot figure
    plt.figure(figsize=(12, 12))
    plt.imshow(image)
    plt.axis('off')
    # Get the current reference
    ax = plt.gca()

    for i in range(num_obj + 1):
        ymin = boxes[i][0]
        xmin = boxes[i][1]
        ymax = boxes[i][2]
        xmax = boxes[i][3]
        # Create a Rectangle patch
        width = image.width*(xmax - xmin)
        height = image.height*(ymax - ymin)
        rect = Rectangle((image.width*xmin, image.height*ymin), width, height, 
                         linewidth=3, edgecolor=edge[i%4], facecolor='none')
        plt.text(image.width*xmin, image.height*ymin, str(i), color='red', 
                 fontdict={"fontsize":20,"fontweight":'bold',"ha":"left", "va":"baseline"})
        # Add the patch to the Axes
        ax.add_patch(rect)
    plt.show()   
    

In [ ]:
object_detect(Path("data") / "jakan.JPG", 0.2)

In [ ]:
object_detect("data\\jakan.JPG", 0.1)

## Pröva med bättre modell

In [ ]:
# Om fel "tf hub OSError: SavedModel file does not exist at" ta bort foldern och kör om...

# Ladda detektorn från TensorFlow Hub
# den här modellen ska ge bättre resultat men är långsammare.
detector = hub.load("https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1")

In [ ]:
object_detect(Path("data") / "jakan.JPG", 0.2)

In [ ]:
object_detect(Path("data") / "jakan.JPG", 0.1)

In [ ]:
object_detect(Path("data") / "hooks.JPG", 0.2)